In [6]:
import tensorflow as tf

# Import Minst data
from helpers.extractor import *
loader = CifarLoader("inputs/datasets/cifar10/", "Grayscale")
#loader = MinstLoader("inputs/datasets/MNIST_data")

Loading files...
Loaded: inputs/datasets/cifar10/data_batch_1
Loaded: inputs/datasets/cifar10/data_batch_2
Loaded: inputs/datasets/cifar10/data_batch_3
Loaded: inputs/datasets/cifar10/data_batch_4
Loaded: inputs/datasets/cifar10/data_batch_5
Loaded: inputs/datasets/cifar10/test_batch


In [7]:
# Parameters
learning_rate = 0.0001
training_iters = 400000
batch_size = 128
display_step = 10

# Network Parameters
n_input = loader.getImageBytes() # MNIST data input (img shape: 28*28)
n_classes = loader.LABELS_COUNT # MNIST total classes (0-9 digits)
img_width = int(loader.getImageWidth())
final_img_width = int((loader.getImageWidth()/2)/2)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

print("image scale: "+str(loader.getImageWidth())+" connected width: "+str(final_img_width))
print(n_input)
print(n_classes)

image scale: 32 connected width: 8
1024
10


In [8]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, img_width, img_width, 1])
    print ("x:", x.get_shape())
    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    print ("conv1:", conv1.get_shape())
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)
    print ("pool1:", conv1.get_shape())

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    print ("conv2:", conv2.get_shape())
    
    conv2 = maxpool2d(conv2, k=2)
    print ("pool2:", conv2.get_shape())
    
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    print ("fc1:", fc1.get_shape())
        
    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    print ("out:", out.get_shape())
     
    return out

In [9]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([final_img_width*final_img_width*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

x: (?, 32, 32, 1)
conv1: (?, 32, 32, 32)
pool1: (?, 16, 16, 32)
conv2: (?, 16, 16, 64)
pool2: (?, 8, 8, 64)
fc1: (?, 1024)
out: (?, 10)


In [10]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch = loader.getNextBatch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch[0], y: batch[1], keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch[0],
                                                              y: batch[1],
                                                              keep_prob: 1.})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " \
            "{:.6f}".format(loss) + ", Training Accuracy= " \
            "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")
    
    
    batch = loader.getTestBatch(256)
    
    # Calculate accuracy for 256 mnist test images
    print ("Testing Accuracy:"+\
        sess.run(accuracy, feed_dict={x: batch[0],
                                      y: batch[1],
                                      keep_prob: 1.}) )
    # Save model weights to disk
    # save_path = saver.save(sess, "digit_conv_nn_graph.ckpt")
    # print "Model saved in file: %s" % save_path

Iter 1280, Minibatch Loss= 44473.296875, Training Accuracy= 0.14844
Iter 2560, Minibatch Loss= 31987.445312, Training Accuracy= 0.11719
Iter 3840, Minibatch Loss= 21846.433594, Training Accuracy= 0.10938
Iter 5120, Minibatch Loss= 18265.001953, Training Accuracy= 0.07031
Iter 6400, Minibatch Loss= 12770.208984, Training Accuracy= 0.11719
Iter 7680, Minibatch Loss= 11558.060547, Training Accuracy= 0.13281
Iter 8960, Minibatch Loss= 11529.963867, Training Accuracy= 0.08594
Iter 10240, Minibatch Loss= 11696.207031, Training Accuracy= 0.07031
Iter 11520, Minibatch Loss= 10527.601562, Training Accuracy= 0.08594
Iter 12800, Minibatch Loss= 11065.783203, Training Accuracy= 0.07812
Iter 14080, Minibatch Loss= 10198.449219, Training Accuracy= 0.11719
Iter 15360, Minibatch Loss= 9674.958008, Training Accuracy= 0.13281
Iter 16640, Minibatch Loss= 10405.151367, Training Accuracy= 0.10938
Iter 17920, Minibatch Loss= 10042.101562, Training Accuracy= 0.19531
Iter 19200, Minibatch Loss= 11032.605469, 

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')